In [6]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras import layers, models

For when doing GPU work

In [7]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# **UNET**

### **Setting up Data**

In [8]:
# Define directories
positive_images_dir = r'archive/Concrete/Positive/Images'
positive_masks_dir = r'archive/Concrete/Positive/Masks'
negative_images_dir = r'archive/Concrete/Negative/Images'
negative_masks_dir = r'archive/Concrete/Negative/Mask'

# Function to load images and masks
def load_images_and_masks(images_dir, masks_dir, target_size=(448, 448)):
    images = []
    masks = []
    
    for filename in os.listdir(images_dir):
        if filename.endswith('.jpg'):
            img_path = os.path.join(images_dir, filename)
            mask_path = os.path.join(masks_dir, filename.replace('.jpg', '.png'))  # Adjust if masks have different format
            
            # Load image
            image = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
            image = tf.keras.preprocessing.image.img_to_array(image) / 447.0  # Normalize
            
            # Load mask
            mask = tf.keras.preprocessing.image.load_img(mask_path, target_size=target_size, color_mode='grayscale')
            mask = tf.keras.preprocessing.image.img_to_array(mask) / 447.0  # Normalize
            
            images.append(image)
            masks.append(mask)
    
    return tf.convert_to_tensor(images), tf.convert_to_tensor(masks)

# Load positive data
positive_images, positive_masks = load_images_and_masks(positive_images_dir, positive_masks_dir, target_size=(448, 448))

# Load negative data
negative_images, negative_masks = load_images_and_masks(negative_images_dir, negative_masks_dir, target_size=(448, 448))

# Combine positive and negative data
images = tf.concat([positive_images, negative_images], axis=0)
masks = tf.concat([positive_masks, negative_masks], axis=0)

# Shuffle the combined data
indices = tf.random.shuffle(tf.range(tf.shape(images)[0]))
images = tf.gather(images, indices)
masks = tf.gather(masks, indices)


FileNotFoundError: [Errno 2] No such file or directory: 'archive/Concrete/Positive/Masks/CRACK500_20160307_164517_1281_361.png'

In [20]:
data = data.map(lambda x,y: (x/227,y))

In [21]:
shuffled_data = data.shuffle(buffer_size=100, reshuffle_each_iteration=False)

In [22]:
training_split_size = int(len(data)*.7)
testing_split_size = int(len(data)*.15)
validation_split_size = int(len(data)*.15)

In [23]:
training_set = shuffled_data.take(training_split_size)
testing_set = shuffled_data.skip(training_split_size).take(testing_split_size)
validation_set = shuffled_data.skip(training_split_size+testing_split_size).take(validation_split_size)

### **Model Buliding**

In [37]:
input_shape = (256, 256, 3)  # Adjust based on your images

# Input layer
inputs = layers.Input(shape=input_shape)

# Encoder
c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
p1 = layers.MaxPooling2D((2, 2))(c1)

c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
p2 = layers.MaxPooling2D((2, 2))(c2)

c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
p3 = layers.MaxPooling2D((2, 2))(c3)

c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
p4 = layers.MaxPooling2D((2, 2))(c4)

# Bottleneck
c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

# Decoder
u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = layers.concatenate([u6, c4])
c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = layers.concatenate([u7, c3])
c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = layers.concatenate([u8, c2])
c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = layers.concatenate([u9, c1])
c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

# Output layer
outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)  # Use softmax for multi-class

# Create the model
model = models.Model(inputs=[inputs], outputs=[outputs])

# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_70 (Conv2D)          (None, 256, 256, 64)         1792      ['input_6[0][0]']             
                                                                                                  
 conv2d_71 (Conv2D)          (None, 256, 256, 64)         36928     ['conv2d_70[0][0]']           
                                                                                                  
 max_pooling2d_20 (MaxPooli  (None, 128, 128, 64)         0         ['conv2d_71[0][0]']           
 ng2D)                                                                                        

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
hist = model.fit(training_set, epochs = 25, validation_data = validation_set)

### **Visualising Results on a sample image**

In [ ]:
import matplotlib.pyplot as plt

# Function to visualize results
def visualize_results(image, true_mask, predicted_mask):
    plt.figure(figsize=(12, 4))
    
    # Original image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title('Original Image')
    plt.axis('off')

    # True mask
    plt.subplot(1, 3, 2)
    plt.imshow(true_mask.squeeze(), cmap='gray')
    plt.title('True Mask')
    plt.axis('off')

    # Predicted mask
    plt.subplot(1, 3, 3)
    plt.imshow(predicted_mask.squeeze(), cmap='gray')
    plt.title('Predicted Mask')
    plt.axis('off')

    plt.show()

# Select a sample image and its true mask
sample_index = 0  # Change this index to visualize different samples
sample_image = images[sample_index].numpy()  # Convert tensor to numpy array
sample_true_mask = masks[sample_index].numpy()  # Convert tensor to numpy array

# Reshape for the model prediction (adding batch dimension)
sample_image_input = np.expand_dims(sample_image, axis=0)

# Predict the mask
sample_predicted_mask = model.predict(sample_image_input)[0]  # Get the first image's prediction

# Threshold the predicted mask for binary segmentation
sample_predicted_mask = (sample_predicted_mask > 0.5).astype(np.float32)

# Visualize the results
visualize_results(sample_image, sample_true_mask, sample_predicted_mask)